In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# 1. carregar
data = fetch_california_housing()
X = data.data          # shape (n_samples, n_features)
y = data.target        # preço mediano da casa

In [ ]:
pd.DataFrame(X, columns=data.feature_names).head()

In [ ]:
# 2. separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# 3. padronizar (rede neural gosta de dados com média 0 e var 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# 4. converter pra tensores do PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [ ]:
input_dim = X_train.shape[1]
print("Formato de treino:", X_train.shape, y_train.shape)

In [ ]:
# definição do modelo
model = nn.Sequential(
    nn.Linear(input_dim, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1)
)
print(model)

In [ ]:
# definição da loss e do otimizador
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# cria dataset e dataloader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# loop de treinamento do modelo
num_epochs = 2000
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    # forward
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # guardar perdas
    train_losses.append(loss.item())

    with torch.no_grad():
        y_test_pred = model(X_test)
        test_loss = loss_fn(y_test_pred, y_test).item()
        test_losses.append(test_loss)

    # printa a loss a cada 20 épocas
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1:03d} | train loss: {loss.item():.4f} | test loss: {test_loss:.4f}")

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(train_losses, label="Treino")
plt.plot(test_losses, label="Teste")
plt.title("Evolução da Loss durante o treinamento")
plt.xlabel("Época")
plt.ylabel("Loss (MSE)")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
with torch.no_grad():
    preds = model(X_test[:5])
print("Predições:", preds.squeeze().numpy())
print("Reais     :", y_test[:5].squeeze().numpy())

In [ ]:
# previsões do modelo no conjunto de teste
with torch.no_grad():
    y_pred_test = model(X_test).numpy()
    y_true_test = y_test.numpy()

# calcular métricas
mse = mean_squared_error(y_true_test, y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true_test, y_pred_test)

print("📈 Avaliação no conjunto de teste:")
print(f" - MSE  (Mean Squared Error):      {mse:.4f}")
print(f" - RMSE (Root Mean Squared Error): {rmse:.4f}")
print(f" - MAE  (Mean Absolute Error):     {mae:.4f}")